In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd

countries_to_keep = [
    "Canada", "Mexico", "United States", "Austria", "Belgium", "France", "Germany", "Ireland", "Luxembourg", "Netherlands",
    "Switzerland", "United Kingdom", "Denmark", "Estonia", "Finland", "Iceland", "Latvia", "Lithuania", "Norway", "Sweden",
    "Greece", "Italy", "Portugal", "Spain", "Turkey", "Czechia", "Hungary", "Poland", "Slovak Republic", "Slovenia",
    "Australia", "Japan", "Korea, Rep.", "New Zealand", "Chile", "Colombia", "Costa Rica"
]

print(len(countries_to_keep))

# Load data (replace 'data.csv' with the actual filename)
df = pd.read_csv('/content/Foreign direct investment in USD-2.csv')  # Add your filename here

# Filter data by countries
df_filtered = df[df['Country Name'].isin(countries_to_keep)]


# # If you want to pivot the table with years as columns (as your comment suggests)
# # First, make sure you have the right columns in place
# df_pivoted = df_filtered.pivot(index=["Country", "Country Code"], columns="Year")

# print(df_pivoted.isnull().sum())

# Save filtered data
df_filtered.to_csv('/content/drive/MyDrive/Capstone Project NY/Cleaned Data/FDI_cleaned.csv')

print(countries_to_keep)
print("Filtered data saved to 'industry_including_construction_value_usd_cleaned.csv'")

37
['Canada', 'Mexico', 'United States', 'Austria', 'Belgium', 'France', 'Germany', 'Ireland', 'Luxembourg', 'Netherlands', 'Switzerland', 'United Kingdom', 'Denmark', 'Estonia', 'Finland', 'Iceland', 'Latvia', 'Lithuania', 'Norway', 'Sweden', 'Greece', 'Italy', 'Portugal', 'Spain', 'Turkey', 'Czechia', 'Hungary', 'Poland', 'Slovak Republic', 'Slovenia', 'Australia', 'Japan', 'Korea, Rep.', 'New Zealand', 'Chile', 'Colombia', 'Costa Rica']
Filtered data saved to 'industry_including_construction_value_usd_cleaned.csv'


In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
# Load the CSV file into a DataFrame
input_file = "/content/drive/MyDrive/Capstone Project NY/Supplemental Data/Import of goods and services as a percentage of GDP.csv"  # Replace with your input file name
df = pd.read_csv(input_file)

# Create a mapping of countries to their country codes
country_code_mapping = {
    "Canada": "CAN",
    "Mexico": "MEX",
    "United States": "USA",
    "Austria": "AUT",
    "Belgium": "BEL",
    "France": "FRA",
    "Germany": "DEU",
    "Ireland": "IRL",
    "Luxembourg": "LUX",
    "Netherlands": "NLD",
    "Switzerland": "CHE",
    "United Kingdom": "GBR",
    "Denmark": "DNK",
    "Estonia": "EST",
    "Finland": "FIN",
    "Iceland": "ISL",
    "Latvia": "LVA",
    "Lithuania": "LTU",
    "Norway": "NOR",
    "Sweden": "SWE",
    "Greece": "GRC",
    "Italy": "ITA",
    "Portugal": "PRT",
    "Spain": "ESP",
    "Turkey": "TUR",
    "Czechia": "CZE",
    "Hungary": "HUN",
    "Poland": "POL",
    "Slovak Republic": "SVK",
    "Slovenia": "SVN",
    "Australia": "AUS",
    "Japan": "JPN",
    "Korea, Rep.": "KOR",
    "New Zealand": "NZL",
    "Chile": "CHL",
    "Colombia": "COL",
    "Costa Rica": "CRI"
}

# Add a new column "Country Code" to the DataFrame
df["Country Code"] = df["Country Name"].map(country_code_mapping)

# Save the updated DataFrame to a new CSV file
output_file = "/content/drive/MyDrive/Capstone Project NY/Supplemental Data/iclean_import_of_goods_and_services_as_a_percentage_of_GDP.csv"  # Replace with your desired output file name
df.to_csv(output_file, index=False)

print(f"Updated CSV file saved as {output_file}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updated CSV file saved as /content/drive/MyDrive/Capstone Project NY/Supplemental Data/iclean_import_of_goods_and_services_as_a_percentage_of_GDP.csv


In [ ]:
print(df['Government Expenditure'].unique())
print(df.isnull().sum())

['Agriculture, forestry, fishing and hunting' 'Communication' 'Defence'
 'Economic affairs n.e.c.' 'Education' 'Environment protection'
 'Fuel and energy' 'General economic, commercial and labour affairs'
 'General public services' 'Health' 'Housing and community amenities'
 'Mining, manufacturing and construction' 'Other industries'
 'Public order and safety' 'RandD Economic affairs'
 'Recreation, culture and religion' 'Social protection' 'Total function'
 'Transport']
Country Name               0
Country Code               0
Government Expenditure     0
2010                      30
2011                      32
2012                      12
2013                       3
2014                       1
2015                       1
2016                       2
2017                       2
2018                       0
2019                       0
2020                       0
2021                      29
dtype: int64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load the dataset
data = pd.read_csv("/content/industry_including_construction_value_usd_cleaned.csv")

# Function to predict missing values and 2023 values
def predict_missing_and_2023(data):
    countries = data['Country'].unique()
    predictions = []

    for country in countries:
        country_data = data[data['Country'] == country]
        years = np.array(range(2010, 2022)).reshape(-1, 1)  # Years 2010-2021
        values = country_data.iloc[:, 2:14].values.flatten()  # Values from 2010-2021

        # Check if there are missing values in 2022
        if pd.isna(country_data['2022'].values[0]):
            # Fit a linear regression model
            model = LinearRegression()
            model.fit(years, values)

            # Predict 2022 and 2023
            pred_2022 = model.predict(np.array([[2022]]))[0]
            pred_2023 = model.predict(np.array([[2023]]))[0]

            # Update the dataset
            data.loc[data['Country'] == country, '2022'] = pred_2022
            data.loc[data['Country'] == country, '2023'] = pred_2023

            # Store predictions
            predictions.append({
                'Country': country,
                '2022': pred_2022,
                '2023': pred_2023
            })
        else:
            # If 2022 is not missing, predict 2023 only
            model = LinearRegression()
            model.fit(years, values)
            pred_2023 = model.predict(np.array([[2023]]))[0]

            # Update the dataset
            data.loc[data['Country'] == country, '2023'] = pred_2023

            # Store predictions
            predictions.append({
                'Country': country,
                '2022': country_data['2022'].values[0],
                '2023': pred_2023
            })

    return data, predictions

# Fill missing values and predict 2023
updated_data, predictions = predict_missing_and_2023(data)

# Save the updated dataset to a new CSV file
updated_data.to_csv("property_tax_%_of_GDP_cleaned_with_predictions.csv", index=False)

# Print predictions
predictions_df = pd.DataFrame(predictions)
print(predictions_df)

            Country      2022      2023
0         Australia  3.143394  3.208814
1           Austria  0.589000  0.585071
2           Belgium  3.276000  3.604881
3            Canada  3.498000  4.000325
4             Chile  1.221000  1.193521
5          Colombia  1.529000  1.753948
6        Costa Rica  0.499000  0.513828
7    Czech Republic  0.184000  0.270731
8           Denmark  1.710000  1.918012
9           Estonia  0.163000  0.162605
10          Finland  1.438000  1.653442
11           France  3.675000  4.127646
12          Germany  1.117000  1.309108
13           Greece  3.255000  3.423795
14          Hungary  0.806000  0.895854
15          Iceland  2.007000  3.610576
16          Ireland  1.021000  0.985267
17            Italy  2.466000  2.607918
18            Japan  2.702000  2.622901
19            Korea  3.796000  4.103865
20           Latvia  0.757000  0.953142
21        Lithuania  0.291000  0.293829
22       Luxembourg  3.691000  4.428621
23           Mexico  0.317000  0.353735
